In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn


In [3]:
import urllib.request
urllib.request.urlretrieve('ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt', 'stations.txt')

('stations.txt', <email.message.Message at 0x110f17320>)

In [3]:
[line for line in open('stations.txt','r')][:10]

['ACW00011604  17.1167  -61.7833   10.1    ST JOHNS COOLIDGE FLD                       \n',
 'ACW00011647  17.1333  -61.7833   19.2    ST JOHNS                                    \n',
 'AE000041196  25.3330   55.5170   34.0    SHARJAH INTER. AIRP            GSN     41196\n',
 'AEM00041194  25.2550   55.3640   10.4    DUBAI INTL                             41194\n',
 'AEM00041217  24.4330   54.6510   26.8    ABU DHABI INTL                         41217\n',
 'AEM00041218  24.2620   55.6090  264.9    AL AIN INTL                            41218\n',
 'AF000040930  35.3170   69.0170 3366.0    NORTH-SALANG                   GSN     40930\n',
 'AFM00040938  34.2100   62.2280  977.2    HERAT                                  40938\n',
 'AFM00040948  34.5660   69.2120 1791.3    KABUL INTL                             40948\n',
 'AFM00040990  31.5000   65.8500 1010.0    KANDAHAR AIRPORT                       40990\n']

In [4]:
stations = {}

for line in open('stations.txt','r'):
    if 'GSN' in line:
        fields = line.split()
    
        stations[fields[0]] = ' '.join(fields[4:])

In [5]:
len(stations)

994

In [6]:
def find_station(s):
    found = {code: name for code, name in stations.items() if s in name}
    
    return found

In [7]:
find_station('LIHUE')

{'USW00022536': 'HI LIHUE WSO AP 1020.1 GSN 91165'}

In [ ]:
url = 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/gsn/USW00022536.dly'
urllib.request.urlretrieve(url,'USW00022536.dly')

In [ ]:
open('USW00022536.dly','r').readlines()[:10]

In [ ]:
def parsefile(filename):
    return np.genfromtxt(filename,
                        delimiter = dly_delimiter,
                        usecols = dly_usecols,
                        dtype = dly_dtype,
                        names = dly_names)

In [ ]:
dly_delimiter = [11, 4, 2, 4] + [5, 1, 1, 1] * 31
dly_usecols = [1, 2, 3] + [4*i for i in range(1, 32)]
dly_dtype = [np.int32, np.int32, (np.str_, 4)] + [np.int32]*31
dly_names = ['year','month', 'obs'] + [str(i) for i in range(1,32)]

In [ ]:
lihue = parsefile('USW00022536.dly')

In [ ]:
def unroll(record):
    startdate = np.datetime64('{}-{:02}'.format(record['year'], record['month']))
    dates = np.arange(startdate, startdate + np.timedelta64(1, 'M'), np.timedelta64(1, 'D'))
    
    rows = [(date, record[str(i+1)]/10) for i, date in enumerate(dates)]
    
    #return np.array(rows)
    return np.array(rows, dtype=[('date', 'M8[D]'), ('value', 'd')])

In [ ]:
def get_obs(fname, obs):
    return np.concatenate([unroll(row) for row in parsefile(fname) if row[2] == obs])

In [ ]:
tmax = get_obs('USW00022536.dly', 'TMAX')
tmax[:10]